# Set Up

In [1]:
from google.colab import drive
import zipfile

drive.mount('/content/drive')

# Get Dataset
with zipfile.ZipFile("/content/drive/MyDrive/ADNI_MANUAL_DATASET.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


Mounted at /content/drive


## Define Config



In [ ]:
IMAGE_SIZE = [200, 200]
CLASS_NAMES = ["AD", "CN", "MCI", "pMCI"]

MODEL_SAVE_PATH = "/content/drive/MyDrive/models"

DATASET_PATH = "/content/dataset/ADNI_MANUAL_DATASET"

## Imports

In [ ]:
import sys
from tensorflow.keras.preprocessing import image_dataset_from_directory
from datetime import datetime
import os
import tensorflow as tf

# Models

In [ ]:
#------------------------------------COGNINET-------------------------------#

def convolutional_block(filters):
    return tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(
            filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(
            filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )


def dense_block(units, dropout_rate):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])

def cogni_net():
    model = tf.keras.Sequential([
            tf.keras.Input(shape=(*IMAGE_SIZE, 1)),

            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
            tf.keras.layers.MaxPool2D(),

            convolutional_block(32),
            convolutional_block(64),

            convolutional_block(128),
            tf.keras.layers.Dropout(0.2),

            convolutional_block(256),
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Flatten(),
            dense_block(512, 0.7),
            dense_block(128, 0.5),
            dense_block(64, 0.3),

            tf.keras.layers.Dense(
                4, activation='softmax')
        ])
    return model

#------------------------------------VGGNET-------------------------------#

def vgg_net():
    input = tf.keras.Input(shape =(*IMAGE_SIZE, 1))
    # 1st Conv Block
    x = tf.keras.layers.Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(input)
    x = tf.keras.layers.Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    # 2nd Conv Block
    x = tf.keras.layers.Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    # 3rd Conv block
    x = tf.keras.layers.Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    # 4th Conv block

    x = tf.keras.layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    # 5th Conv block

    x = tf.keras.layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size =2, strides =2, padding ='same')(x)

    # Fully connected layers
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(units = 4096, activation ='relu')(x)
    x = tf.keras.layers.Dense(units = 4096, activation ='relu')(x)
    output = tf.keras.layers.Dense(units = 4, activation ='softmax')(x)

    model = tf.keras.Model (inputs=input, outputs =output)

    return model

#------------------------------------DENSENET-------------------------------#


def bn_rl_conv(x,filters,kernel=1,strides=1):

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
    return x

def dense_block(x, repetition, filters=32):

   for _ in range(repetition):
        y = bn_rl_conv(x, 4*filters)
        y = bn_rl_conv(y, filters, 3)
        x = tf.keras.layers.concatenate([y,x])
   return x

def transition_layer(x):

    x = bn_rl_conv(x, tf.keras.backend.int_shape(x)[-1] //2 )
    x = tf.keras.layers.AvgPool2D(2, strides = 2, padding = 'same')(x)
    return x

def dense_net():
    input = tf.keras.Input(shape =(*IMAGE_SIZE, 1))
    x = tf.keras.layers.Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = tf.keras.layers.MaxPool2D(3, strides = 2, padding = 'same')(x)

    for repetition in [6,12,24,16]:
        d = dense_block(x, repetition)
        x = transition_layer(d)

    x = tf.keras.layers.GlobalAveragePooling2D()(d)
    output = tf.keras.layers.Dense(4, activation = 'softmax')(x)

    model = tf.keras.Model(input, output)
    return model

# ------------------------------------RESNET-------------------------------#

def residual_module(data,
                    filters,
                    stride,
                    reduce=False,
                    reg=0.0001,
                    bn_eps=2e-5,
                    bn_momentum=0.9):
    bn_1 = tf.keras.layers.BatchNormalization(axis=-1,
                              epsilon=bn_eps,
                           momentum=bn_momentum)(data)
    act_1 = tf.keras.layers.ReLU()(bn_1)
    conv_1 = tf.keras.layers.Conv2D(filters=int(filters / 4.),
                    kernel_size=(1, 1),
                    use_bias=False,
                    kernel_regularizer=tf.keras.regularizers.l2(reg))(act_1)

    bn_2 = tf.keras.layers.BatchNormalization(axis=-1,
                              epsilon=bn_eps,
                         momentum=bn_momentum)(conv_1)
    act_2 = tf.keras.layers.ReLU()(bn_2)
    conv_2 = tf.keras.layers.Conv2D(filters=int(filters / 4.),
                    kernel_size=(3, 3),
                    strides=stride,
                    padding='same',
                    use_bias=False,
                    kernel_regularizer=tf.keras.regularizers.l2(reg))(act_2)
    bn_3 = tf.keras.layers.BatchNormalization(axis=-1,
                              epsilon=bn_eps,
                              momentum=bn_momentum)(conv_2)
    act_3 = tf.keras.layers.ReLU()(bn_3)
    conv_3 = tf.keras.layers.Conv2D(filters=filters,
                    kernel_size=(1, 1),
                    use_bias=False,
                    kernel_regularizer=tf.keras.regularizers.l2(reg))(act_3)

    if reduce:
        shortcut = tf.keras.layers.Conv2D(filters=filters,
                          kernel_size=(1, 1),
                          strides=stride,
                          use_bias=False,
                          kernel_regularizer=tf.keras.regularizers.l2(reg))(act_1)

    x = tf.keras.layers.Add()([conv_3, shortcut])
    return x

def resnet(input_shape,
                 classes,
                 stages,
                 filters,
                 reg=1e-3,
                 bn_eps=2e-5,
                 bn_momentum=0.9):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.BatchNormalization(axis=-1,
                           epsilon=bn_eps,
                           
                         momentum=bn_momentum)(inputs)
    x = tf.keras.layers.Conv2D(filters[0], (3, 3),
               use_bias=False,
               padding='same',
                kernel_regularizer=tf.keras.regularizers.l2(reg))(x)
    
    for i in range(len(stages)):
        stride = (1, 1) if i == 0 else (2, 2)
        x = residual_module(data=x,
                            filters=filters[i + 1],
                            stride=stride,
                            reduce=True,
                            bn_eps=bn_eps,
                            bn_momentum=bn_momentum)
        for j in range(stages[i] - 1):
            x = residual_module(data=x,
                                filters=filters[i + 
                                               1],
                                stride=(1, 1),
                                bn_eps=bn_eps,
                                
                            bn_momentum=bn_momentum)
            
    x = tf.keras.layers.BatchNormalization(axis=-1,
                           epsilon=bn_eps,
                           momentum=bn_momentum)(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.AveragePooling2D((8, 8))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(classes, kernel_regularizer=l2(reg))(x)
    x = tf.keras.layers.Softmax()(x)
    return tf.keras.Model(inputs, x, name='resnet')

# Hardware Setup

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Limit GPU memory growth
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        pass

# Main Model Trainer

In [ ]:
STRATEGY = tf.distribute.get_strategy()
AUTOTUNE = tf.data.AUTOTUNE

BATCH_SIZE = 16 * STRATEGY.num_replicas_in_sync
EPOCHS = 50

MODEL_NAME = "DENSENET"


base_dir = f"{MODEL_SAVE_PATH}/{MODEL_NAME}-{datetime.now().strftime('%Y-%m-%d-%H:%M')}"
os.makedirs(base_dir, exist_ok=True)

with open(f"{base_dir}/log.log", "w") as sys.stdout:
  print("REPLICAS: ", STRATEGY.num_replicas_in_sync)

  train_ds = image_dataset_from_directory(
      DATASET_PATH,
      labels="inferred",
      label_mode="categorical",
      image_size=IMAGE_SIZE,
      color_mode="grayscale",
      validation_split=0.2,
      subset="training",
      seed=1337
  ).prefetch(buffer_size=AUTOTUNE)

  validation_ds = image_dataset_from_directory(
      DATASET_PATH,
      labels="inferred",
      label_mode="categorical",
      image_size=IMAGE_SIZE,
      color_mode="grayscale",
      validation_split=0.2,
      subset="validation",
      seed=1337
  ).prefetch(buffer_size=AUTOTUNE)

  with STRATEGY.scope():
      model = dense_net()

  model.summary()

  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



  # Defining Callbacks
  save_best = tf.keras.callbacks.ModelCheckpoint(
      filepath=f"{base_dir}/model.h5", monitor='val_loss', save_best_only=True)
  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
      patience=10, monitor='val_accuracy', factor=0.6, min_lr=0.0000001)

  history = model.fit(
      train_ds,
      validation_data=validation_ds,
      epochs=EPOCHS,
      callbacks=[save_best, reduce_lr]
  )


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
